# Execution Benchmarking

In this module, we will look at how threading and multiprocesses perform on multiple differnt tasks.

In [1]:
from utils.timer import DecoTimer

import os
import requests
import uuid
from queue import Queue
import logging

from threading import Thread
from multiprocessing.pool import Pool


In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### 1. Download Images

In [3]:
num_of_pics = 20
image_endpoints = [f'https://picsum.photos/800/800?image={i}' for i in range(num_of_pics)]
filenames = [str(uuid.uuid4())[:8] for _ in range(num_of_pics)]

def download_image(image_url, filename):
    # Set stream to True to prevent the requests library suck memory 
    # and stays around 30kb regardless size of the download file
    img_data = requests.get(image_url, stream = True, headers={'Cache-Control': 'no-cache'}).content
    with open(f'images/{filename}.bmp', 'wb') as handler:
        handler.write(img_data)
        logger.info(f'writing {filename}')

def create_image_dir():
    os.system("mkdir -p images")

def remove_images():
    os.system("rm images/*")

download_dir = create_image_dir()

#### 1. 1 Single Threaded

In [4]:

with DecoTimer("Download Image - Single Thread"):
    for image_endpoint, filename in zip(image_endpoints, filenames):
        download_image(image_endpoint, filename)        


>>>>Starting Function Download Image - Single Thread...


INFO:__main__:writing 5be15764
INFO:__main__:writing 7a080e0c
INFO:__main__:writing faadb65d
INFO:__main__:writing 1acbc16b
INFO:__main__:writing 7ee5fd5d
INFO:__main__:writing aeb98553
INFO:__main__:writing f1042790
INFO:__main__:writing 0a5dec33
INFO:__main__:writing 49d1ac73
INFO:__main__:writing 769aec01
INFO:__main__:writing 53f0c3a0
INFO:__main__:writing 08f0c7b9
INFO:__main__:writing 0637533a
INFO:__main__:writing 95c8ef0d
INFO:__main__:writing 02778389
INFO:__main__:writing e490523c
INFO:__main__:writing 1c2ddbcd
INFO:__main__:writing bf481c9b
INFO:__main__:writing 1d8c4f49
INFO:__main__:writing efdc51da


<<Finished function Download Image - Single Thread in 22.773748874664307 seconds


#### 1.2 Multithreading Example

In [10]:
class DownloadWorker(Thread):
    
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue # create a worker queue to pass in the image filename sequentially
        
    def run(self):
        while True:
            image_endpoint, filename = self.queue.get()
            try:
                download_image(image_endpoint, filename)
            finally:
                self.queue.task_done()

with DecoTimer("Download Image - Multithreading with 4 Workers"):
    queue = Queue()
    for x in range(4):
        worker = DownloadWorker(queue)
        worker.daemon = True # enable daemon to allow main thread exit even when workers are blocking
        worker.start()
    # send in filenames to workers from main thread
    for image_endpoint, filename in zip(image_endpoints, filenames):
        queue.put((image_endpoint, filename))
    queue.join()

>>>>Starting Function Download Image - Multithreading with 4 Workers...


INFO:__main__:writing 7a080e0c
INFO:__main__:writing faadb65d
INFO:__main__:writing 1acbc16b
INFO:__main__:writing 7ee5fd5d
INFO:__main__:writing f1042790
INFO:__main__:writing 5be15764
INFO:__main__:writing aeb98553
INFO:__main__:writing 0a5dec33
INFO:__main__:writing 08f0c7b9
INFO:__main__:writing 53f0c3a0
INFO:__main__:writing 49d1ac73
INFO:__main__:writing 769aec01
INFO:__main__:writing 95c8ef0d
INFO:__main__:writing 0637533a
INFO:__main__:writing 02778389
INFO:__main__:writing e490523c
INFO:__main__:writing 1c2ddbcd
INFO:__main__:writing bf481c9b
INFO:__main__:writing efdc51da
INFO:__main__:writing 1d8c4f49


<<Finished function Download Image - Multithreading with 4 Workers in 2.589384078979492 seconds


#### 1.3 Multiprocessing Example

In [6]:

with DecoTimer("Download Image - Multiprocessing with 4 Workers"):
    args = list(zip(image_endpoints, filenames))
    with Pool(4) as p:
        p.starmap(download_image, args)


>>>>Starting Function Download Image - Multiprocessing with 4 Workers...


INFO:__main__:writing 7ee5fd5d
INFO:__main__:writing f1042790
INFO:__main__:writing faadb65d
INFO:__main__:writing 1acbc16b
INFO:__main__:writing aeb98553
INFO:__main__:writing 5be15764
INFO:__main__:writing 0a5dec33
INFO:__main__:writing 7a080e0c
INFO:__main__:writing 53f0c3a0
INFO:__main__:writing 08f0c7b9
INFO:__main__:writing 0637533a
INFO:__main__:writing 49d1ac73
INFO:__main__:writing 02778389
INFO:__main__:writing 1c2ddbcd
INFO:__main__:writing 95c8ef0d
INFO:__main__:writing e490523c
INFO:__main__:writing bf481c9b
INFO:__main__:writing 769aec01
INFO:__main__:writing 1d8c4f49
INFO:__main__:writing efdc51da


<<Finished function Download Image - Multiprocessing with 4 Workers in 2.8966729640960693 seconds


### 2. Load Image and convert it to Numpy Array

<i>Make sure you have the images downloaded in the images/ directory from the previous step.</i>

In [7]:
from PIL import Image
import numpy as np

def image_read_as_nparray(filename):
    image = Image.open(f'images/{filename}.bmp')
    logger.info(f'loading image {filename}')
    return np.array(image.getdata()).reshape(image.size[0], image.size[1], 3)

#### 2.1 Single Threaded Example

In [8]:
with DecoTimer("Loading Images and Convert it to Numpy Array - Single Threaded Example"):
    images = [image_read_as_nparray(filename) for filename in filenames]


INFO:__main__:loading image 5be15764


>>>>Starting Function Loading Images and Convert it to Numpy Array - Single Threaded Example...


INFO:__main__:loading image 7a080e0c
INFO:__main__:loading image faadb65d
INFO:__main__:loading image 1acbc16b
INFO:__main__:loading image 7ee5fd5d
INFO:__main__:loading image aeb98553
INFO:__main__:loading image f1042790
INFO:__main__:loading image 0a5dec33
INFO:__main__:loading image 49d1ac73
INFO:__main__:loading image 769aec01
INFO:__main__:loading image 53f0c3a0
INFO:__main__:loading image 08f0c7b9
INFO:__main__:loading image 0637533a
INFO:__main__:loading image 95c8ef0d
INFO:__main__:loading image 02778389
INFO:__main__:loading image e490523c
INFO:__main__:loading image 1c2ddbcd
INFO:__main__:loading image bf481c9b
INFO:__main__:loading image 1d8c4f49
INFO:__main__:loading image efdc51da


<<Finished function Loading Images and Convert it to Numpy Array - Single Threaded Example in 13.100675106048584 seconds


#### 2.2 Multithreading Example

In [11]:
class LoadingWorker(Thread):
    
    def __init__(self, in_queue, out_queue):
        Thread.__init__(self)
        self.in_queue = in_queue # create a worker queue to pass in the image filename sequentially
        self.out_queue = out_queue # creaet a worker queue to store the output data
        
    def run(self):
        while True:
            filename = self.in_queue.get()
            try:
                self.out_queue.put(image_read_as_nparray(filename))
            finally:
                self.in_queue.task_done()

with DecoTimer("Loading Images and Convert it to Numpy Array - Multithreading Example"):
    in_queue = Queue()
    out_queue = Queue()
    for x in range(4):
        worker = LoadingWorker(in_queue, out_queue)
        worker.daemon = True
        worker.start()
    
    for filename in filenames:
        in_queue.put((filename))
    in_queue.join()
    
    images = []
    while out_queue.qsize() > 0:
        try:
            images.append(out_queue.get())
        finally:    
            out_queue.task_done()
    out_queue.join()

        
    
    

INFO:__main__:loading image 5be15764
INFO:__main__:loading image faadb65d
INFO:__main__:loading image 1acbc16b
INFO:__main__:loading image 7a080e0c


>>>>Starting Function Loading Images and Convert it to Numpy Array - Multithreading Example...


INFO:__main__:loading image f1042790
INFO:__main__:loading image 0a5dec33
INFO:__main__:loading image aeb98553
INFO:__main__:loading image 7ee5fd5d
INFO:__main__:loading image 49d1ac73
INFO:__main__:loading image 769aec01
INFO:__main__:loading image 53f0c3a0
INFO:__main__:loading image 0637533a
INFO:__main__:loading image 95c8ef0d
INFO:__main__:loading image 02778389
INFO:__main__:loading image 08f0c7b9
INFO:__main__:loading image e490523c
INFO:__main__:loading image 1c2ddbcd
INFO:__main__:loading image bf481c9b
INFO:__main__:loading image 1d8c4f49
INFO:__main__:loading image efdc51da


<<Finished function Loading Images and Convert it to Numpy Array - Multithreading Example in 14.157184839248657 seconds


#### 2.3 Multiprocessing Example

In [48]:
with DecoTimer("Load Images and Convert it to Numpy Array - Multiprocessing Example"):
    with Pool(4) as p:
        images = p.map(image_read_as_nparray, filenames)
        

>>>>Starting Function Load Images and Convert it to Numpy Array - Multiprocessing Example...


INFO:__main__:loading image 5be15764
INFO:__main__:loading image 7ee5fd5d
INFO:__main__:loading image faadb65d
INFO:__main__:loading image f1042790
INFO:__main__:loading image aeb98553
INFO:__main__:loading image 0a5dec33
INFO:__main__:loading image 7a080e0c
INFO:__main__:loading image 1acbc16b
INFO:__main__:loading image 49d1ac73
INFO:__main__:loading image 53f0c3a0
INFO:__main__:loading image 769aec01
INFO:__main__:loading image 0637533a
INFO:__main__:loading image 02778389
INFO:__main__:loading image 08f0c7b9
INFO:__main__:loading image 95c8ef0d
INFO:__main__:loading image e490523c
INFO:__main__:loading image 1c2ddbcd
INFO:__main__:loading image 1d8c4f49
INFO:__main__:loading image bf481c9b
INFO:__main__:loading image efdc51da


<<Finished function Load Images and Convert it to Numpy Array - Multiprocessing Example in 10.23697805404663 seconds


### 3. Grayscale Image

In [49]:
def grayscale_for_loop(image, n):
    for i in range(len(image)):
        for j in range(len(image[0])):
            average = (image[i][j][0] + image[i][j][1] + image[i][j][2]) / 3
            image[i][j][0] = image[i][j][1] = image[i][j][2] = average
    im = Image.fromarray(np.uint8(image))
    logging.info('Grayscaled image')
    if n == 0:
        im.show() # only display one image to prove that the image is grayscaled
    return im

In [50]:
# Save Original Images for quick resetting
original_images = [np.copy(image) for image in images]
Image.fromarray(np.uint8(images[0])).show()

#### 3.1 Single Threaded (using For-Loop) 48-53 seconds

In [51]:
with DecoTimer("Grayscale Image - Single Threaded For Loop"):
    for n, image in enumerate(images):
        im = grayscale_for_loop(image, n)

>>>>Starting Function Grayscale Image - Single Threaded For Loop...


INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image


<<Finished function Grayscale Image - Single Threaded For Loop in 49.26708245277405 seconds


#### Multithreading Example (using For-Loop) 54-58 seconds

In [52]:
# Resetting Images
images = [np.copy(original_image) for original_image in original_images]
Image.fromarray(np.uint8(images[0])).show()

In [53]:
class GrayscaleWorker(Thread):
    
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue # create a worker queue to pass in numpy array sequentially
        
    def run(self):
        while True:
            n, image = self.queue.get()
            try:
                im = grayscale_for_loop(image, n)
            finally:
                self.queue.task_done()

with DecoTimer("Grayscale Image - Multithreading For Loop"):
    queue = Queue()
    for x in range(4):
        worker = GrayscaleWorker(queue)
        worker.daemon = True
        worker.start()
    
    for n, image in enumerate(images):
        queue.put((n, image))
    queue.join()

>>>>Starting Function Grayscale Image - Multithreading For Loop...


INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image


<<Finished function Grayscale Image - Multithreading For Loop in 54.78285098075867 seconds


#### Multiprocessing Example (using For-Loop) 26-30 seconds

In [56]:
# Resetting Images
images = [np.copy(original_image) for original_image in original_images]
Image.fromarray(np.uint8(images[0])).show()

In [57]:

with DecoTimer("Grayscale Image - MultiProcessing Example "):
    with Pool(4) as p:
        im_arr = p.starmap(grayscale_for_loop, zip(images, [i for i in range(len(images))]))
        

>>>>Starting Function Grayscale Image - MultiProcessing Example ...


INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image


<<Finished function Grayscale Image - MultiProcessing Example  in 26.930018186569214 seconds


#### 2.4 Numpy Vector Operations (Single Thread, Eliminate Inner Loops) 48~56 seconds

In [58]:
# Resetting Images
images = [np.copy(original_image) for original_image in original_images]
Image.fromarray(np.uint8(images[0])).show()

In [59]:
def grayscale_vector(image, n):
    average = ((image[:, :, 0] + image[:, :, 1] + image[:, :, 2]) / 3).astype(int)
    image[:, :, 0] = image[:, :, 1] = image[:, :, 2] = average
    im = Image.fromarray(np.uint8(image)).show()
    logging.info('Grayscaled image')
    if n == 0:
        im.show()
    return im

In [60]:
with DecoTimer("Single Threaded with Numpy Vectorization, Eliminate Inner Loop"):
    for n, image in enumerate(images):
        im = grayscale_for_loop(image, n)

>>>>Starting Function Single Threaded with Numpy Vectorization, Eliminate Inner Loop...


INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image
INFO:root:Grayscaled image


<<Finished function Single Threaded with Numpy Vectorization, Eliminate Inner Loop in 53.031378984451294 seconds


#### 2.5 Numpy Vector Operation (Single Thread, Vectorized Across All Images):    **Less than 1 Second**

In [65]:
# Resetting Images
images = [np.copy(original_image) for original_image in original_images]
Image.fromarray(np.uint8(images[0])).show()

In [66]:
with DecoTimer("Vectorized Across All Images"):
    npimages = np.array(images)
    average = ((npimages[:, :, :, 0] + npimages[:, :, :, 1] + npimages[:, :, :, 2]) / 3).astype(int)
    npimages[:, :, :, 0] = npimages[:, :, :, 1] = npimages[:, :, :, 2] = average
    
    for n in range(len(npimages)):
        im = Image.fromarray(np.uint8(npimages[n]))
        if n == 0:
            im.show()


>>>>Starting Function Vectorized Across All Images...
<<Finished function Vectorized Across All Images in 0.8148548603057861 seconds


In [67]:
remove_images() # clean up